# Final Project - Dashboard

**Name:** Omar Alejandro Guzmán Munguía

**e-mail:** omar.guzman5063@alumnos.udg.mx

# MODULES

In [ ]:
import panel as pn
pn.extension()

import pandas as pd
import numpy as np

import panel.widgets as pnw
pn.extension('plotly')

# FUNCTIONS

# WIDGETS

# PANEL